In [0]:
import json
from sentence_transformers import SentenceTransformer
from azure.storage.blob import BlobServiceClient

# Conectar al Blob Storage
blob_service_client = BlobServiceClient.from_connection_string("***************************************")
container_name = "*****"
input_blob_name = "articles.json"
embeddings_blob_name = "articles_embeddings_sbert_cls_ALL_roberta.json"
input_blob_client = blob_service_client.get_blob_client(container_name, input_blob_name)
embeddings_blob_client = blob_service_client.get_blob_client(container_name, embeddings_blob_name)

# Descargar el archivo JSON
try:
    downloaded_blob = input_blob_client.download_blob().readall()
    articles_data = json.loads(downloaded_blob)
    print(f"Downloaded {len(articles_data)} articles.")
except Exception as e:
    print(f"Error al descargar el archivo JSON: {e}")
    raise

# Cargar el modelo all-roberta-large-v1
model_name = "all-roberta-large-v1"
model = SentenceTransformer(model_name)

# Función para generar embeddings [CLS]
def generate_cls_embedding(text):
    embedding = model.encode(text, convert_to_tensor=True)
    return embedding.cpu().tolist()

# Procesar todos los artículos y generar embeddings
embeddings_data = []

for idx, article in enumerate(articles_data):
    try:
        content_id = article['contentId']
        text = article['text']  
        embeddings = generate_cls_embedding(text)
        embeddings_data.append({
            'contentId': content_id,
            'embeddings': embeddings
        })

        if idx % 100 == 0:
            print(f"Processed {idx + 1} articles")

    except Exception as e:
        print(f"Error al procesar el artículo {content_id}: {e}")

# Guardar los embeddings generados en un archivo JSON
try:
    embeddings_json = json.dumps(embeddings_data, ensure_ascii=False, indent=4)
    embeddings_blob_client.upload_blob(embeddings_json, overwrite=True)
    print(f"Uploaded embeddings for {len(embeddings_data)} articles to {embeddings_blob_name}.")
except Exception as e:
    print(f"Error al subir los embeddings al Blob Storage: {e}")
    raise


/databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:167: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
 original_result = python_builtin_import(name, globals, locals, fromlist, level)
Downloaded 2097 articles.
modules.json: 0%| | 0.00/349 [00:00<?, ?B/s]
config_sentence_transformers.json: 0%| | 0.00/116 [00:00<?, ?B/s]
README.md: 0%| | 0.00/9.89k [00:00<?, ?B/s]
sentence_bert_config.json: 0%| | 0.00/53.0 [00:00<?, ?B/s]
/databricks/python/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
 warnings.warn(
config.json: 0%| | 0.00/650 [00:00<?, ?B/s]
model.safetensors: 0%| | 0.00/1.42G [00:00<?, ?B/s]
tokenizer_config.json: 0%| | 0.00/328 [00:00<?, ?B/s]
vocab.json: 0%| | 0.00/798k [00:00<?, ?B/s]
merges.txt: 0%| | 0.00/456k [00:00<?, ?B/s]
tokenizer.json: 0%| | 0.00/1.36M [00:00<?, ?B/s]
special_tokens_map.json: 0%| | 0.00/239 [00:00<?, ?B/s]
1_Pooling/config.json: 0%| | 0.00/191 [00:00<?, ?B/s]
Processed 1 articles
Processed 101 articles
Processed 201 articles
Processed 301 articles
Processed 401 articles
Processed 501 articles
Processed 601 articles
Processed 701 articles
Processed 801 articles
Processed 901 articles
Processed 1001 articles
Processed 1101 articles
Processed 1201 articles
Processed 1301 articles
Processed 1401 articles
Processed 1501 articles
Processed 1601 articles
Processed 1701 articles
Processed 1801 articles
Processed 1901 articles
Processed 2001 articles
Uploaded embeddings for 2097 articles to articles_embeddings_sbert_cls_ALL_roberta.json.